In [72]:
example = """
.#.
..#
###
""".strip().splitlines()

with open('day17.txt', 'r') as f:
    data = f.readlines()

## Strategy
We're using an immutable `State` construct which is transformed according to a set of rules. This allows us to snapshot the state for evaluation before we move to the next step.

We have the challenge of needing to introduce neighbouring cubes (which may start in an active state depending on the number of neighbours) so it is necessary to first build up
a set of the new state's cube space and then iterating to the new state.

In [91]:
from typing import Iterator

class Cube(object):
    def __init__(self, x: int, y: int, z: int = 0, active: bool = False):
        self.x = x
        self.y = y
        self.z = z
        self.active = active

    @property
    def index(self) -> str:
        return Cube.format_index(self.x, self.y, self.z)

    @staticmethod
    def format_index(*coords: int) -> str:
        return ",".join(map(str, coords))

    def neighbours(self, state: "State") -> Iterator[Cube]:
        d = [-1, 0, 1]

        for dx in d:
            for dy in d:
                for dz in d:
                    if dx == dy == dz == 0:
                        continue

                    default_cube = Cube(self.x + dx, self.y + dy, self.z + dz)
                    yield state.get(default_cube.index, default_cube)

    def active_neighbours(self, state: "State") -> int:
        return sum(1 for neighbour in self.neighbours(state) if neighbour.active)

    def step(self, state: "State") -> Cube:
        active_neighbours = self.active_neighbours(state)

        if self.active:
            return self.with_state(active_neighbours in [2, 3])
        else:
            return self.with_state(active_neighbours == 3)

    def with_state(self, active: bool) -> Cube:
        return Cube(self.x, self.y, self.z, active=active)

    def __eq__(self, other: Cube) -> bool:
        return self.index == other.index
        
    def __ne__(self, other: Cube) -> bool:
        return self.index != other.index

    def __hash__(self):
        return hash((self.x, self.y, self.z))

In [92]:
from typing import Iterable

class State(object):
    def __init__(self, cubes: Iterable[Cube]):
        self.cubes: Dict[str, Cube] = {
            cube.index: cube
            for cube in cubes if cube.active
        }

        self.x_range = (min(cube.x for cube in self.cubes.values()), max(cube.x for cube in self.cubes.values()))
        self.y_range = (min(cube.y for cube in self.cubes.values()), max(cube.y for cube in self.cubes.values()))
        self.z_range = (min(cube.z for cube in self.cubes.values()), max(cube.z for cube in self.cubes.values()))

    def transform(self) -> "State":
        new_cube_space = set(neighbour for cube in self.cubes.values() for neighbour in cube.neighbours(self))
        return State(cube.step(self) for cube in new_cube_space)

    def get(self, index: str, default: Cube) -> Cube:
        if index in self.cubes:
            return self.cubes[index]

        return default

    def print(self):
        for z in range(self.z_range[0], self.z_range[1] + 1):
            print(f"z={z}")
            for y in range(self.y_range[0], self.y_range[1] + 1):
                print("".join('#' if self.get(x, y, z).active else '.' for x in range(self.x_range[0], self.x_range[1] + 1)) + f"  | y = {y}")

            print("\n")


In [93]:
from typing import Iterable, Dict, Type

class EnergySource(object):
    def __init__(self, init: Iterable[Iterable[str]], cube_type: Type[Cube] = Cube):
        self.state = State(cube_type(x, y, active=(state == '#')) for y, line in enumerate(init) for x, state in enumerate(line))

    def boot(self, print_state: int = 0):
        if print_state > 0:
            print("Before any cycles:\n")
            self.state.print()

        for i in range(0, 6):

            self.step()
            if print_state > i:
                print(f"\nAfter {i + 1} cycle{'s' if i != 0 else ''}")
                self.state.print()

    def step(self):
        self.state = self.state.transform()

    def active_cubes(self) -> int:
        return sum(1 for cube in self.state.cubes.values() if cube.active)

example_source = EnergySource(example)
example_source.boot(print_state=0)
assert example_source.active_cubes() == 112

source = EnergySource(data)
source.boot()
print(f"Active Cubes (Part 1): {source.active_cubes()}")

Active Cubes (Part 1): 380


In [94]:
class HyperCube(Cube):
    def __init__(self, x: int, y: int, z: int = 0, w: int = 0, active: bool = False):
        super().__init__(x, y, z, active)
        self.w = w

    @property
    def index(self) -> str:
        return Cube.format_index(self.x, self.y, self.z, self.w)

    def neighbours(self, state: "State") -> Iterator["HyperCube"]:
        d = [-1, 0, 1]

        for dx in d:
            for dy in d:
                for dz in d:
                    for dw in d:
                        if dx == dy == dz == dw == 0:
                            continue

                        default_cube = HyperCube(self.x + dx, self.y + dy, self.z + dz, self.w + dw)
                        yield state.get(default_cube.index, default_cube)

    def with_state(self, active: bool) -> Cube:
        return HyperCube(self.x, self.y, self.z, self.w, active=active)

source = EnergySource(data, cube_type=HyperCube)
source.boot()
print(f"Active Cubes (Part 2): {source.active_cubes()}")

Active Cubes (Part 2): 2332
